In [17]:
import pandas as pd
import talib
import os
import logging
import time
import numpy as np
import datetime as dt
import backtrader as bt
import pandas as pd
import itertools
from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Tradimo
%matplotlib qt

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("study")
handler = logging.FileHandler('study.log')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [12]:
stock_data_store = pd.HDFStore(r'E:\project\trade_engine\data_processing\quandl_data_processor\stock_day.h5')

# 取出所有volume>3000000的股票的symbol组成一个list
symbol_list = pd.read_csv(
    r"E:\project\trade_engine\data_processing\quandl_data_processor\filter_stock_symbols.csv").loc[
              :, "symbols"].tolist()

# 用一个字典来把所有股票数据全部读到内存中去.
symbol_frame_dict = {}
for symbol in symbol_list:
    # 从hdf中把数据读进内存中
    symbol_frame = stock_data_store.select('day', where=['symbol=="{}"'.format(symbol)])
    # 我们只研究2000年后的股票数据, 之前的就不要了
    symbol_frame = symbol_frame[symbol_frame.index > dt.datetime(2000, 1, 1)]
    # 把数据缓存到一个字典中, symbol作为key
    symbol_frame_dict[symbol] = symbol_frame

In [42]:
# 首先要定义一个param list
param_name_list = ["m", "n", "min_price", "float_factor", "high_factor"]

# 在此定义策略中的所有参数取值范围, 用list数据结构
param_value_selections = [
    [20],
    [30],
    [5],
    [0.09],
    [0.3]
]


# 循环所有的参数可能的组合
for param_values in itertools.product(*param_value_selections):
    # 把当前所有params的取值放在一个字典中
    param = {}
    for index, param_name in enumerate(param_name_list):
        param[param_name] = param_values[index]

In [43]:
param

{'m': 20, 'n': 30, 'min_price': 5, 'float_factor': 0.09, 'high_factor': 0.3}

In [80]:
# 创建一个list用于记录计算出的所有股票的buy signal data frame
buy_signal_list = []

# 创建一个list用于记录计算出的所有股票的buy signal data frame
buy_signal_list = []
for symbol in ["AAPL"]:
    # 首先从内存中取出数据
    data = symbol_frame_dict[symbol]

    # 下面计算一些指标
    close = data.loc[:, "adj_close"]
    adj_close_change = (close - close.shift(0)) / close.shift(1)
    close_change_m_days_max = close.shift(0).rolling(param["m"]).max()
    close_change_m_days_max_percent = (close_change_m_days_max - close) / close

    close_change_m_days_min = close.shift(0).rolling(param["m"]).min()
    close_change_m_days_min_percent = (close_change_m_days_min - close) / close

    previous_high = close.shift(0).rolling(param["m"] + param["n"]).max()
    previous_high_percent = (previous_high - close) / close

    close_greater_than_min_price = (close > param["min_price"]) * 1

    # 根据指标计算买入信号.
    buy_signal = (
            (close_change_m_days_max_percent < param["float_factor"])
            & (close_change_m_days_min_percent > -param["float_factor"])
            & (previous_high_percent > param["high_factor"])
            & (close_greater_than_min_price == 1)
            #& (close > close_change_m_days_max)
    )

    # 计算未来多少天的收益值

    next_7_days_avg_close_change = adj_close_change.rolling(7).mean().shift(-7)
    next_14_days_avg_close_change = adj_close_change.rolling(14).mean().shift(-14)
    next_21_days_avg_close_change = adj_close_change.rolling(21).mean().shift(-21)
    next_28_days_avg_close_change = adj_close_change.rolling(28).mean().shift(-28)
    buy_sig_index = buy_signal[buy_signal == True].index
    
    buy_signal_frame = pd.concat([
        data.loc[buy_sig_index],
        next_7_days_avg_close_change.loc[buy_sig_index].rename("next_7_days_avg_close_change"),
        next_14_days_avg_close_change.loc[buy_sig_index].rename("next_14_days_avg_close_change"),
        next_21_days_avg_close_change.loc[buy_sig_index].rename("next_21_days_avg_close_change"),
        next_28_days_avg_close_change.loc[buy_sig_index].rename("next_28_days_avg_close_change")],
        axis = 1
    )

In [81]:
buy_signal_frame

,symbol,unadj_open,unadj_high,unadj_low,unadj_close,unadj_vol,dividends,splits,adj_open,adj_high,adj_low,adj_close,adj_volume,next_7_days_avg_close_change,next_14_days_avg_close_change,next_21_days_avg_close_change,next_28_days_avg_close_change
date,,,,,,,,,,,,,,,,,
2006-07-10,AAPL,55.7000,56.4900,54.50,55.0000,18905200.0,0.0,1.0,6.887928,6.985620,6.739534,6.801365,132336400.0,0.0,0.0,0.0,0.0
2008-02-28,AAPL,127.2000,132.2000,125.77,129.9100,57794800.0,0.0,1.0,15.729702,16.348008,15.552867,16.064824,404563600.0,0.0,0.0,0.0,0.0
2008-02-29,AAPL,129.7500,130.2100,124.80,125.0200,44838600.0,0.0,1.0,16.045038,16.101922,15.432915,15.460121,313870200.0,0.0,0.0,0.0,0.0
2008-03-03,AAPL,124.4400,125.9800,118.00,121.7300,56894400.0,0.0,1.0,15.388397,15.578835,14.592019,15.053275,398260800.0,0.0,0.0,0.0,0.0
2008-03-04,AAPL,121.9900,124.8800,120.40,124.6200,63763700.0,0.0,1.0,15.085427,15.442808,14.888806,15.410656,446345900.0,0.0,0.0,0.0,0.0
2008-03-05,AAPL,123.5800,125.1400,122.25,124.4899,43637000.0,0.0,1.0,15.282049,15.474960,15.117579,15.394568,305459000.0,0.0,0.0,0.0,0.0
2008-03-06,AAPL,124.4899,127.5000,120.81,120.9300,52632100.0,0.0,1.0,15.394568,15.766800,14.939507,14.954347,368424700.0,0.0,0.0,0.0,0.0
2008-03-07,AAPL,120.4100,122.9799,119.05,122.2500,43945100.0,0.0,1.0,14.890043,15.207840,14.721863,15.117579,307615700.0,0.0,0.0,0.0,0.0
2008-03-10,AAPL,121.9800,123.4600,119.37,119.6900,35699600.0,0.0,1.0,15.084191,15.267209,14.761435,14.801007,249897200.0,0.0,0.0,0.0,0.0


画图

In [172]:
cerebro = bt.Cerebro()
data = bt.feeds.PandasData(
    dataname=data,
    datetime=None,
    open=8,
    high=9,
    low=10,
    close=11,
    volume=12,
    openinterest=-1
)
cerebro.adddata(data)
cerebro.run()

In [173]:
cerebro.plot(style='candle', iplot=False)

[[<Figure size 640x480 with 4 Axes>]]

In [174]:
# b = Bokeh(style='bar') # 黑底, 单页
b = Bokeh(style='bar', tabs='multi') # 黑底, 多页
# b = Bokeh(style='bar', scheme=Tradimo()) # 传统白底, 单页
# b = Bokeh(style='bar', tabs='multi', scheme=Tradimo()) # 传统白底, 多页
cerebro.plot(b, iplot=False)

[[<backtrader_plotting.bokeh.bokeh.FigurePage at 0x1d79eb8bf60>]]